In [1]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:128"
import torch
import numpy as np
import pandas as pd
import pickle
import argparse

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

c:\Users\sonng\anaconda3\envs\tsff_env\lib\site-packages\pytorch_forecasting\models\base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import warnings
# To ignore all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", ".*does not have many workers.*")
warnings.filterwarnings("ignore", ".*and is already saved during checkpointing*")
warnings.filterwarnings("ignore", ".*The number of training batches*")

import logging
logging.getLogger("lightning").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.ERROR)


In [3]:
from ESRNN.m4_data import *
from ESRNN.utils_evaluation import evaluate_prediction_owa
from ESRNN.utils_visualization import plot_grid_prediction

In [4]:
# setting parameters
batch_size = 64
patience = 100
max_epochs = 200

In [5]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name="Hourly",
                                                               directory="../data/M4",
                                                               num_obs=414)

In [6]:
unique_ids = y_train_df['unique_id'].unique()

In [7]:
all_forecasts = {}

In [8]:
pl.seed_everything(42)

Seed set to 42


42

In [9]:
for unique_id in unique_ids:

    print(f'Currently training: {unique_id}')

    # Filter data for the current series (train and val data)
    df = y_train_df[y_train_df['unique_id'] == unique_id].copy()
    df['ds'] = (df['ds'] - df['ds'].min()).dt.total_seconds() // 3600
    df['ds'] = df['ds'].astype(int)
    df_train_val = df.iloc[-24*7:].reset_index(drop=True)
    
    # Test data
    df = y_test_df.drop(columns=['y_hat_naive2'])[y_test_df['unique_id'] == unique_id].copy()
    df['ds'] = (df['ds'] - df['ds'].min()).dt.total_seconds() // 3600 + df_train_val['ds'].max() + 1  #700
    df['ds'] = df['ds'].astype(int)
    df_test = df.reset_index(drop=True)

    # Create the TimeSeriesDataSet for training
    max_encoder_length = 24*7
    max_prediction_length = 48

    training = TimeSeriesDataSet(
        df_train_val.iloc[:-max_prediction_length],
        time_idx="ds",
        target="y",
        group_ids=['unique_id'],
        max_encoder_length=max_encoder_length,
        # min_encoder_length=max_encoder_length // 2,
        min_encoder_length=1,
        max_prediction_length=max_prediction_length,
        # min_prediction_length=max_prediction_length // 2,
        min_prediction_length=1,
        target_normalizer=GroupNormalizer(
            groups=["unique_id"], transformation="softplus"
        ),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        # allow_missing_timesteps=Truex
        )
    
    validation = TimeSeriesDataSet.from_dataset(training, df_train_val, predict=True, stop_randomization=True)

    # creating the test data that includes the encoder and decoder data
    encoder_data = df_train_val[lambda x: x.ds > x.ds.max() - max_encoder_length]
    df_test.y = df_train_val.y[df_train_val.ds == df_train_val.ds.max()].values[0]
    decoder_data = df_test
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

    batch_size = 64  # set this between 32 to 128
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    # test_dataloader = test.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

    # loading the best parameters from best_params folder
    with open(os.path.join('../code/best_params', f"study_{unique_id}.pkl"), 'rb') as fin:
        study = pickle.load(fin)
    best_params = study.best_trial.params

    # configure network and trainer
    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=patience, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="gpu",
        gradient_clip_val=best_params['gradient_clip_val'],
        limit_train_batches=50,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[early_stop_callback],
        logger=False,
        enable_model_summary=False,
        enable_checkpointing=False
    )

    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=best_params['learning_rate'],
        hidden_size=best_params['hidden_size'],
        attention_head_size=best_params['attention_head_size'],
        dropout=best_params['dropout'],
        hidden_continuous_size=best_params['hidden_continuous_size'],
        loss=SMAPE(),
        # log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        optimizer="Ranger",
        reduce_on_plateau_patience=4,
    )

    trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,

    )

    # predictions = tft.predict(test_dataloader, return_y=False, trainer_kwargs=dict(accelerator="gpu"))
    new_raw_predictions = tft.predict(new_prediction_data, mode="raw", return_x=True, trainer_kwargs=dict(accelerator="gpu"))
    all_forecasts[unique_id] = new_raw_predictions.output.prediction.cpu().numpy().flatten()



Currently training: H1


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 136: 100%|██████████| 3/3 [00:00<00:00,  5.25it/s, train_loss_step=0.149, val_loss=0.0947, train_loss_epoch=0.153]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Currently training: H10
Epoch 102: 100%|██████████| 3/3 [00:00<00:00,  5.15it/s, train_loss_step=0.0509, val_loss=0.0667, train_loss_epoch=0.0514]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Currently training: H392
Epoch 113: 100%|██████████| 3/3 [00:00<00:00,  5.86it/s, train_loss_step=0.499, val_loss=0.493, train_loss_epoch=0.516]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Currently training: H393
Epoch 102: 100%|██████████| 3/3 [00:00<00:00,  5.93it/s, train_loss_step=0.285, val_loss=0.178, train_loss_epoch=0.279]

In [6]:
y_hat = pd.read_csv('../results/m4/TFT/test/y_hat_df_tft_standalone_test.csv')

In [7]:
y_hat

,unique_id,y_hat
0,H1,622.84090
1,H1,562.16364
2,H1,516.36316
3,H1,485.46240
4,H1,462.48798
...,...,...
19867,H99,20267.65200
19868,H99,20323.19100
19869,H99,20378.58200
19870,H99,20433.81800


In [9]:
y_hat_df = X_test_df.drop(columns='x')

In [11]:
y_hat_df['y_hat'] = y_hat['y_hat']

In [12]:
y_hat_df

,unique_id,ds,y_hat
0,H1,1970-01-30 04:00:00,622.84090
1,H1,1970-01-30 05:00:00,562.16364
2,H1,1970-01-30 06:00:00,516.36316
3,H1,1970-01-30 07:00:00,485.46240
4,H1,1970-01-30 08:00:00,462.48798
...,...,...,...
19867,H99,1970-01-31 23:00:00,20267.65200
19868,H99,1970-02-01 00:00:00,20323.19100
19869,H99,1970-02-01 01:00:00,20378.58200
19870,H99,1970-02-01 02:00:00,20433.81800


In [13]:
evaluate_prediction_owa(y_hat_df, y_train_df, X_test_df, y_test_df, naive2_seasonality=24)

===============  Model evaluation  ==============
OWA: 1.383 
SMAPE: 24.442 
MASE: 3.438 


(1.3825838096415182, 3.438210427153768, 24.44211616438968)